In [5]:
import pandas as pd
import torch
import numpy as np

# Load CSV file
data = pd.read_csv('/Users/dhruvkulshrestha/Desktop/PIML_DS/lorenze_attractor.csv')

# Extract time and states
t = data['t'].values  # Time points (using 't' as the time column)
states = data[['X', 'Y', 'Z']].values  # States (X, Y, Z)

# Convert to PyTorch tensors
t = torch.tensor(t, dtype=torch.float32).reshape(-1, 1)
states = torch.tensor(states, dtype=torch.float32)

# Split into training and validation sets
train_size = int(0.8 * len(t))
t_train, t_val = t[:train_size], t[train_size:]
states_train, states_val = states[:train_size], states[train_size:]

In [6]:
print(data.head())

   Unnamed: 0         X         Y          Z       U        t
0           0  0.000000  1.000000  20.000000  0.0000  0.00000
1           1  0.002497  0.999753  19.986685  0.0005  0.00025
2           2  0.004986  0.999510  19.973366  0.0010  0.00050
3           3  0.007469  0.999273  19.960056  0.0015  0.00075
4           4  0.009945  0.999041  19.946756  0.0020  0.00100


In [24]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print("Using device:", device)

t_train = t_train.to(torch.float32).to(device)
t_val = t_val.to(torch.float32).to(device)
states_train = states_train.to(torch.float32).to(device)
states_val = states_val.to(torch.float32).to(device)

Using device: mps


In [25]:
sigma = torch.tensor(10.0, dtype=torch.float32, device=device)
rho = torch.tensor(28.0, dtype=torch.float32, device=device)
beta = torch.tensor(8.0 / 3.0, dtype=torch.float32, device=device)

In [26]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
from torchdiffeq import odeint  # This is MPS-compatible now with float32

# Make sure sigma, rho, beta are already defined as float32 tensors on the device

class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(1, 64), nn.Tanh(),
            nn.Linear(64, 64), nn.Tanh(),
            nn.Linear(64, 3)  # Output: [x, y, z]
        )

    def forward(self, t):
        # Ensure t is float32 and on the right device
        t = t.to(torch.float32).to(next(self.parameters()).device)
        return self.net(t)

    def compute_residuals(self, t):
        t = t.clone().detach().requires_grad_(True).to(torch.float32).to(next(self.parameters()).device)
        pred = self(t)

        x, y, z = pred[:, 0], pred[:, 1], pred[:, 2]

        x_t = torch.autograd.grad(x.sum(), t, create_graph=True)[0]
        y_t = torch.autograd.grad(y.sum(), t, create_graph=True)[0]
        z_t = torch.autograd.grad(z.sum(), t, create_graph=True)[0]

        residuals = torch.stack([
            x_t - sigma * (y - x),
            y_t - (x * (rho - z) - y),
            z_t - (x * y - beta * z)
        ], dim=1)

        return residuals

In [27]:
class NeuralODE(nn.Module):
    def __init__(self):
        super(NeuralODE, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(3, 64), nn.Tanh(),
            nn.Linear(64, 64), nn.Tanh(),
            nn.Linear(64, 3)
        )
    
    def forward(self, t, y):
        # Ensure y is float32 and on the same device as model
        y = y.to(torch.float32).to(next(self.parameters()).device)
        return self.net(y)

In [28]:
pinn = PINN().to(device)  # Moves model to MPS (or CPU fallback)
neural_ode = NeuralODE().to(device)

optimizer_pinn = torch.optim.Adam(pinn.parameters(), lr=1e-3)
optimizer_ode = torch.optim.Adam(neural_ode.parameters(), lr=1e-3)

In [29]:
num_epochs = 1000

# Ensure values are float32 before torch.linspace
t_min = t_train.min().item()
t_max = t_train.max().item()

t_colloc = torch.linspace(t_min, t_max, 100, dtype=torch.float32).reshape(-1, 1).to(device)

# Re-confirm float32 + device (safe redundancy)
t_train = t_train.to(torch.float32).to(device)
states_train = states_train.to(torch.float32).to(device)

In [31]:
for epoch in range(num_epochs):
    # === PINN Training ===
    optimizer_pinn.zero_grad()
    
    init_loss = torch.mean((pinn(t_train[0:1]) - states_train[0]) ** 2)
    residuals = pinn.compute_residuals(t_colloc)
    physics_loss = torch.mean(residuals ** 2)
    
    loss_pinn = init_loss + physics_loss
    loss_pinn.backward()
    optimizer_pinn.step()

    # === Neural ODE Training ===
    optimizer_ode.zero_grad()
    
    y0 = states_train[0].to(torch.float32).to(device)
    t_input = t_train[:, 0].to(torch.float32).to(device)

    
    pred_ode = odeint(
        neural_ode,
        y0,
        t_input,
        rtol=1e-5,
        atol=1e-5,
        options={"dtype": torch.float32}
    )

    loss_ode = torch.mean((pred_ode - states_train) ** 2)
    loss_ode.backward()
    optimizer_ode.step()

    # === Logging ===
    if epoch % 100 == 0:
        print(f'Epoch {epoch}, PINN Loss: {loss_pinn.item():.4f}, ODE Loss: {loss_ode.item():.4f}')


Epoch 0, PINN Loss: 137.4544, ODE Loss: 110.2679


KeyboardInterrupt: 